In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import nltk
nltk.download('stopwords')
nltk.download('punkt')


from functions import *

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [65]:
# if data not saved as csv, run this
import os
if not os.path.exists('data/merged_titles_labels.csv'):
    df1 = pd.read_csv('../eda/small1/labeled.csv')
    df2 = pd.read_csv('../eda/small2/labeled.csv')
    df3 = pd.read_csv('../eda/small3/labeled.csv')
    df = pd.concat([df1, df2, df3], ignore_index=True).reset_index(drop=True)
    df.to_csv('data/merged_titles_labels.csv', index=False)
    df.head()
else:
    df = pd.read_csv('data/merged_titles_labels.csv')
df.head()

,title,is_clickbait
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",0
2,Why the Truth Might Get You Fired,1
3,15 Civilians Killed In Single US Airstrike Hav...,1
4,Iranian woman jailed for fictional unpublished...,1


In [66]:
df_temp.iloc[376]['title']

'Nuclear tensions between US, Russia reach ‘dangerous’ point'

In [67]:
%run functions.py
if not os.path.exists('data/preprocessed_titles_labels.pkl'):
    df = preprocess_title(df, verbose=True)
    df.to_pickle('data/preprocessed_titles_labels.pkl') 

else:
    df = pd.read_pickle('data/preprocessed_titles_labels.pkl')
df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/adammajczyk/nltk_data...
[nltk_data]   Package words is already up-to-date!


0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        15 Civilians Killed In Single US Airstrike Hav...
4        Iranian woman jailed for fictional unpublished...
                               ...                        
75017    To Make Female Hearts Flutter in Iraq, Throw a...
75018    British Liberal Democrat Patsy Calton, 56, die...
75019    Drone smartphone app to help heart attack vict...
75020    Netanyahu Urges Pope Benedict, in Israel, to D...
75021    Computer Makers Prepare to Stake Bigger Claim ...
Name: title, Length: 75022, dtype: object
Removing numbers and replacing with words...
0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                        Why the Truth Might Get You Fired
3        fifteen Civilians Killed In Single US Airstrik...
4        Iranian woman jaile

/Users/adammajczyk/Sem7/inzynierka/pracaInzynierska/modelling/functions.py:259: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].apply(lambda x: [word for word in x if word not in stop_words])


,title,is_clickbait
0,"[house, dem, aide, even, see, comey, letter, j...",1
1,"[flynn, hillary, clinton, big, woman, campus, ...",0
2,"[truth, might, get, fired]",1
3,"[fifteen, civilian, killed, single, usa, airst...",1
4,"[iranian, woman, jailed, fictional, unpublishe...",1


In [72]:
from tqdm.notebook import tqdm
import time
# VECTOR_SIZE = 2500
# WINDOW = 4
EPOCHS = 500

# EPOCHS = 100
WORKERS = 16
MIN_COUNT = 1
# train many word2vec models with diferent VECTOR_SIZE and WINDOW
VECTOR_SIZEs = [
    100, 
    250, 
    500, 
    1000, 
    # 1500, 
    # 2000, 
    # 2500, 
    # 3000
]

WINDOWs = [3, 4, 5, 6, 7, 8]
SGs = [0, 1]


##################################################
# # --uncomment for sample model training--
EPOCHS = 200
VECTOR_SIZEs = [500]
WINDOWs = [4]
SGs = [0]
##################################################


print('Start training')
# sleep 200 ms
time.sleep(0.2)

for VECTOR_SIZE in tqdm(VECTOR_SIZEs):
    print(f'Current VECTOR_SIZE: {VECTOR_SIZE}')
    for WINDOW in tqdm(WINDOWs, desc=f'WINDOW'):
        for sg in tqdm(SGs, desc=f'SG'):
            model = Word2Vec(df['title'], vector_size=VECTOR_SIZE, window=WINDOW, min_count=MIN_COUNT, workers=WORKERS, sg=sg)
            model.train(df['title'], total_examples=len(df['title']), epochs=EPOCHS)
            model.save(f'word2vec_models/word2vec_vs{VECTOR_SIZE}_win{WINDOW}_sg{sg}.model')

Start training


  0%|          | 0/1 [00:00<?, ?it/s]

Current VECTOR_SIZE: 500


WINDOW:   0%|          | 0/1 [00:00<?, ?it/s]

SG:   0%|          | 0/1 [00:00<?, ?it/s]

In [73]:
model.wv.most_similar('trump')

[('new', 0.6627725958824158),
 ('time', 0.604753851890564),
 ('york', 0.5681480169296265),
 ('breitbart', 0.5589354038238525),
 ('usa', 0.558771550655365),
 ('donald', 0.5273823142051697),
 ('obama', 0.5060986876487732),
 ('hillary', 0.5046316385269165),
 ('clinton', 0.4971393942832947),
 ('two', 0.43150410056114197)]

In [74]:
# save vocabulary
with open('data/vocabulary.txt', 'w') as f:
    for word in model.wv.index_to_key:
        f.write(word + '\n')